In [10]:
import os
import cv2
import numpy as np
import pydicom
from pydicom import dcmread
from pydicom.pixel_data_handlers.util import apply_voi_lut
from pathlib import Path
import re # regular expression

DICOM Path

In [11]:
data_path ='/mnt/cafetera/substrate/KO_bluecare' # path to the ok images from cullera (until Feb, 2024)
dicom_dir = str(data_path+'/DICOMDIR')
print(dicom_dir)

/mnt/cafetera/substrate/KO_bluecare/DICOMDIR


Output PNG Path

In [12]:
output_path = '/home/habtamu/data/substrate/KO_bluecare'

if not os.path.exists(output_path):
    os.makedirs(output_path)

In [14]:
#resize_img= 0.5 # resize image
threshold_img = False # threshold and crop

# the hospital which acquired the images (b for Bluecare, c for Cullera)
hospital = 'b'

Read DICOM Directory

In [15]:
ds = pydicom.dcmread(dicom_dir)
root_dir = Path(ds.filename).resolve().parent
print(f'Root directory: {root_dir}\n')

Root directory: /mnt/cafetera/substrate/KO_bluecare



Check wether the input scan intensities are inverted or not based on DICOM information

In [16]:
def check_dcm_inversion(dcm_in):
    """
    - PresentationLUTShape == 'INVERTED'
    - PhotometricIntepretation = 'MONOCHROME'
    """
    inverted = False

    try:
        if dcm_in.PresentationLUTShape == 'INVERSE':
            inverted = True
    except:
        pass

    try:
        if dcm_in.PhotometricInterpretation == 'MONOCHROME1':
            inverted = True
    except:
        pass

    return inverted

Extracts first numbers of a string. Needs regular expression (re)

In [17]:
# Original: 008989-YARA-003459
# Extracted: 008989
def extract_first_number_token(input_string):
    # Use regular expression to find the first token of numbers
    match = re.search(r'\d+', input_string)
    
    if match:
        return match.group()
    else:
        return None

Threshold and crop images

In [18]:
def deal_DICOM(filename, folder):
    found = False
    dcm_img = pydicom.dcmread(str(filename))
    
    # print all the DICOM tags
    for element in dcm_img:
        print(element)
    
    print ('->',str(filename), 'PatientID ', dcm_img.PatientID, 'Modality', dcm_img.Modality)
    
    if 'Image Storage' in dcm_img.SOPClassUID.name:
        img = dcm_img.pixel_array
        if 'WindowWidth' in dcm_img:
            print('Dataset has windowing')
        img = apply_voi_lut(dcm_img.pixel_array, dcm_img)
        
        # Convert to uint
        img = ((img / img.max()) * 255).astype('uint8')

        # Fix if the image intensities are inverted
        if check_dcm_inversion(dcm_img):
            img = ~img

        # Extract first numbers of the PatientID
        patient_id = extract_first_number_token(dcm_img.PatientID)
        print('Patient ID:', patient_id)
        print('Filename:', filename)

        if patient_id is None: 
            #patient_id = filename
            return found

        # Filename for images from Cullera
        out_fn = output_path+'/'+folder+'/'+ patient_id + '_' + os.path.basename(filename)+'_'+ hospital +'.png'
        print (out_fn)
        
        # resize image
        #img = cv2.resize(img, (0,0), fx=resize_img, fy=resize_img)

        # threshold and crop the image
        if (threshold_img):      
            threshold_value = 100
            # Apply the threshold to the image
            img_th = np.where(img > threshold_value, 255, 0).astype(np.uint8)
            img_th = ~img_th

            # now cut image. 
            # removing spurious pixels at the edges.
            ival = 10 
            img_th = img_th[ival:-ival, ival:-ival]

            contours, _ = cv2.findContours(img_th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            # Initialize variables to store the coordinates of the outer bounding box
            x_min = float('inf')
            y_min = float('inf')
            x_max = -1
            y_max = -1

            # Iterate through the contours and find the coordinates of the outer bounding box
            for contour in contours:
                x, y, w, h = cv2.boundingRect(contour)

                # Update the coordinates of the outer bounding box
                x_min = min(x_min, x)
                y_min = min(y_min, y)
                x_max = max(x_max, x + w)
                y_max = max(y_max, y + h)

            print(y_min, y_max, x_min,x_max)
            img = img[y_min+ival:y_max-ival, x_min+ival:x_max-ival]

        # save image
        cv2.imwrite(out_fn,img)   
        found = True       
    return found 

Patient -> Study -> Series -> Images

In [ ]:
# Iterate through the PATIENT records
for patient in ds.patient_records:
    print(
        f"PATIENT: PatientID={patient.PatientID}, "
        f"PatientName={patient.PatientName}"
    )

    # Find all the STUDY records for the patient
    studies = [
        ii for ii in patient.children if ii.DirectoryRecordType == "STUDY"
    ]
    for study in studies:
        descr = study.StudyDescription or "(no value available)"
        print(
            f"{'  ' * 1}STUDY: StudyID={study.StudyID}, "
            f"StudyDate={study.StudyDate}, StudyDescription={descr}"
        )

        # Find all the SERIES records in the study
        all_series = [
            ii for ii in study.children if ii.DirectoryRecordType == "SERIES"
        ]
        for series in all_series:
            # Find all the IMAGE records in the series
            images = [
                ii for ii in series.children
                if ii.DirectoryRecordType == "IMAGE"
            ]
            plural = ('', 's')[len(images) > 1]

            descr = getattr(
                series, "SeriesDescription", "(no value available)"
            )
            print(
                f"{'  ' * 2}SERIES: SeriesNumber={series.SeriesNumber}, "
                f"Modality={series.Modality}, SeriesDescription={descr} - "
                f"{len(images)} SOP Instance{plural}"
            )

            # Get the absolute file path to each instance
            # Each IMAGE contains a relative file path to the root directory
            elems = [ii["ReferencedFileID"] for ii in images]
            
            # Make sure the relative file path is always a list of str
            paths = [[ee.value] if ee.VM == 1 else ee.value for ee in elems]
            paths = [Path(*p) for p in paths]

            # List the instance file paths
            for p in paths:
                print(f"{'  ' * 3}IMAGE: Path={os.fspath(p)}")
                deal_DICOM(Path(root_dir) / p , 'prova')